In [1]:
import pandas as pd
from datetime import datetime

def duration_intersection(time, flag_time, minutes):
    increment_minute = 1 if (time - flag_time).total_seconds() % 60 != 0 else 0
    tarification_start_time_minutes = (time - flag_time).total_seconds() // 60 + increment_minute
    tarification_end_time_minutes = minutes - tarification_start_time_minutes
    return tarification_end_time_minutes, tarification_start_time_minutes , 0

def bonus_duration_intersection(time, flag_time, minutes):
    increment_minute = 1 if (time - flag_time).total_seconds() % 60 != 0 else 0
    tarification_start_time_minutes = (time + pd.Timedelta(minutes=5) - flag_time).total_seconds() // 60 + increment_minute
    tarification_end_time_minutes = 5 - tarification_start_time_minutes
    bonus_minutes = minutes - 5
    return tarification_end_time_minutes, tarification_start_time_minutes, bonus_minutes
    

def compute_minute_types(start, end):
    minutes = ((end - start).total_seconds() -1) // 60
    tarification_start_time = start.replace(hour=8, minute=0, second=0)
    tarification_end_time = start.replace(hour=16, minute=0, second=0)
    
    tarification_end_time_minutes, tarification_start_time_minutes, bonus_minutes = 0,0,0
    
    if minutes > 5:
        if start < tarification_start_time or start >= tarification_end_time:
            # tarification_end_time from tarification_start_time
            if end > tarification_start_time and end < tarification_end_time:
                if start + pd.Timedelta(minutes=5) > tarification_start_time:
                    return bonus_duration_intersection(start , tarification_start_time , minutes)  
            tarification_end_time_minutes = 5
            bonus_minutes = minutes - 5

        elif start > tarification_start_time and start < tarification_end_time:
            #from tarification_start_time to tarification_end_time
            if end > tarification_end_time:
                if start + pd.Timedelta(minutes=5) > tarification_end_time:
                    return bonus_duration_intersection(start , tarification_end_time , minutes) 
            tarification_start_time_minutes = 5
            bonus_minutes = minutes - 5
            
    else:
        if start < tarification_start_time or start >= tarification_end_time:
            if end > tarification_start_time and end < tarification_end_time:
                # in case we remaining seconds we count 1 more minute
                return duration_intersection(end, tarification_start_time, minutes)
            tarification_end_time_minutes = minutes
        elif start > tarification_start_time and start < tarification_end_time:
            #from tarification_start_time to tarification_end_time
            if end > tarification_end_time:
                return duration_intersection(tarification_end_time, start, minutes)
            tarification_start_time_minutes = minutes
            
    return tarification_end_time_minutes, tarification_start_time_minutes, bonus_minutes

# Define the column names
column_names = ['number', 'start', 'end']

# Read the CSV file into a DataFrame with custom column names
df = pd.read_csv("generated_sample_2.csv", header=None, names=column_names)

most_frequent_number = df['number'].value_counts().idxmax()

# Convert "start" and "end" columns to datetime
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

# Find the most frequent number
most_frequent_number = df['number'].value_counts().idxmax()

# Initialize the total cost
total_cost = 0 

for index, row in df.iterrows():
    if row['number'] == most_frequent_number:
        continue
    tarification_start_time,tarification_end_time,bonus=compute_minute_types(row["start"], pd.to_datetime(row["end"]))
    total_cost += 0.5*tarification_end_time + 1*tarification_start_time + 0.2*bonus
    
print("Total cost:", total_cost, "CZK")
    

Total cost: 663.3 CZK
